#Plotly Python Open Source Graphing Library

Plotly's Python graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
Plotly.py is free and open source and you can view the source, report issues or contribute on GitHub.https://plotly.com/python/

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<iframe width="509" height="407" src="https://www.youtube.com/embed/p8t5cOjlEPU" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

Frank Sinatra: Love and Marriage - YouTube

Lyrics:

Love and marriage, love and marriage/They go together like a horse and carriage/This I tell you, brother/You can't have one without the other/Love and marriage, love and marriage/It's an institute you can't disparage/Ask the local gentry
And they will say it's elementary/Try, try, try to separate them/It's an illusion/Try, try, try, and you will only come
To this conclusion/Love and marriage, love and marriage/Go together like a horse and carriage/Dad was told by mother
You can't have one, you can't have none/You can't have one without the other/Try, try, try to separate them/It's an illusion/Try, try, try, and you will only come/To this conclusion/Love and marriage, love and marriage
They go together like the horse and carriage/Dad was told by mother/You can't have one, you can't have none
You can't have one without the other/No sir

Source: LyricFind
Songwriters: James Van Heusen / Sammy Cahn

Love & Marriage lyrics © Peermusic Publishing, Warner Chappell Music, Inc, Concord Music Publishing LLC

In [ ]:
df=pd.read_csv("../input//married-at-first-sight/mafs.csv")
df.head()

In [ ]:
corr=df[df.columns.sort_values()].corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

fig = go.Figure(data=go.Heatmap(z=corr.mask(mask),
                                x=corr.columns.values,
                                y=corr.columns.values,
                                xgap=1, ygap=1,
                                colorscale="Rainbow",
                                colorbar_thickness=20,
                                colorbar_ticklen=3,
                                zmid=0),
                layout = go.Layout(title_text='Correlation Matrix', template='plotly_dark',
                height=900,
                xaxis_showgrid=False,
                yaxis_showgrid=False,
                yaxis_autorange='reversed'))
fig.show()

In [ ]:
def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 8 , 6 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )

plot_correlation_map(df) 

In [ ]:
fig = px.box(df, x="Couple", y="Age", points="all")
fig.show()

#Thanks to rossinEndrew: SHAP VALUES 
https://www.kaggle.com/endrewrossin/fast-initial-lightgbm-model-to-detect-exam-result/comments

In [ ]:
import shap
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import random

In [ ]:
df.isnull().sum()

In [ ]:
df['Location'] = df['Location'].replace(['Decision','DrJosephCilona'], [0,1])

In [ ]:
SEED = 99
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
dfmodel = df.copy()

# read the "object" columns and use labelEncoder to transform to numeric
for col in dfmodel.columns[dfmodel.dtypes == 'object']:
    le = LabelEncoder()
    dfmodel[col] = dfmodel[col].astype(str)
    le.fit(dfmodel[col])
    dfmodel[col] = le.transform(dfmodel[col])

In [ ]:
#change columns names to alphanumeric
dfmodel.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in dfmodel.columns]

In [ ]:
X = dfmodel.drop(['Location','Couple'], axis = 1)
y = dfmodel['Location']

In [ ]:
lgb_params = {
                    'objective':'binary',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.005,
                    'num_leaves': 20,
                    'max_depth':-1,
                    'subsample':0.9,
                    'n_estimators':2500,
                    'seed': SEED,
                    'early_stopping_rounds':100, 
                }

In [ ]:
# choose the number of folds, and create a variable to store the auc values and the iteration values.
K = 5
folds = KFold(K, shuffle = True, random_state = SEED)
best_scorecv= 0
best_iteration=0

# Separate data in folds, create train and validation dataframes, train the model and cauculate the mean AUC.
for fold , (train_index,test_index) in enumerate(folds.split(X, y)):
    print('Fold:',fold+1)
          
    X_traincv, X_testcv = X.iloc[train_index], X.iloc[test_index]
    y_traincv, y_testcv = y.iloc[train_index], y.iloc[test_index]
    
    train_data = lgb.Dataset(X_traincv, y_traincv)
    val_data   = lgb.Dataset(X_testcv, y_testcv)
    
    LGBM = lgb.train(lgb_params, train_data, valid_sets=[train_data,val_data], verbose_eval=250)
    best_scorecv += LGBM.best_score['valid_1']['auc']
    best_iteration += LGBM.best_iteration

best_scorecv /= K
best_iteration /= K
print('\n Mean AUC score:', best_scorecv)
print('\n Mean best iteration:', best_iteration)

Final Model Modify the hyperparameters to use the best iteration value and train the final model

In [ ]:
lgb_params = {
                    'objective':'binary',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.05,
                    'num_leaves': 20,
                    'max_depth':-1,
                    'subsample':0.9,
                    'n_estimators':round(best_iteration),
                    'seed': SEED,
                    'early_stopping_rounds':None, 
                }

train_data_final = lgb.Dataset(X, y)
LGBM = lgb.train(lgb_params, train_data)

In [ ]:
print(LGBM)

In [ ]:
# telling wich model to use
explainer = shap.TreeExplainer(LGBM)
# Calculating the Shap values of X features
shap_values = explainer.shap_values(X)

In [ ]:
shap.summary_plot(shap_values[1], X, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values[1], X)

In [ ]:
fig = px.scatter_matrix(df, dimensions=["Couple", "Season", "Age", "RachelDeAlto"], color="Location")
fig.show()

I declare my LOVE to PYTHON and PLOTLY. Since both are friendly even for those with lack of programming languages.

Like me.

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxITEhUTExQVFhUXGRcYGRgYGBgYHRkaFxgYFxsZGRoYHSggGholGxcVITEiJSkrLi4uFx8zODMtNygtLisBCgoKDg0OGxAQGy0mICYwLy8tLy0tLS8vLy0tLS0tLS8tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLf/AABEIAKgBLAMBEQACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAAAAgMEBQYBBwj/xABFEAACAQIEAwUEBQkIAQUBAAABAhEAAwQSITEFQVEGEyJhcTKBkbEHcqHB8BQjM0JSssLR4RU0U2JzgpLxJDWTorPTY//EABsBAQADAQEBAQAAAAAAAAAAAAACAwQBBQYH/8QAOhEAAgECBAQEBQIFBAEFAAAAAAECAxEEEiExBRNBUSIyYXGBscHR8DORBhRCofEjNFLhFRYkNUNy/9oADAMBAAIRAxEAPwD28jX8edAECgCBQEDhjmXV3V2DHQCIHKs2Hk7yjKak0+nRPZME+BWkBAoBvvkzZJGbeKr50OZy7+K17dbdztna4pxt6irDh3L6UAZfSgAj0roAD0rgDL6UBX8Ue4B4BoASSI5ctaz1ua2lD9y2EqcIuU9bdDLX8bcuEmZUAxFwAglDBgHUA8o86ywVRyvO7T/NCurUU8mVWRtMHrbQmJyr8hXpB7j0ChwIFAYz6RO2qcO7gAZndwSn/wDIEZ29eS+c9K9DAYB4nN0SX9+gNbg8Ql22ly2QyOoZSOYIkGsMouDcZboD0CogIFAJA1Pu++gG8av5t+XhbbTl1FSh5kclsxy0ogeg8/nXHuFsKgVw6ECgCBQBAoBLjb1oBRWgILEmw5aCcr7CNgR18qsslNW9CbSUtBzGjwD69r/7ErkN/g/kyB2+Pzlv/d+7SPlZwdF5MxT9YCYjlXMrtc6N4UeK59f+FaS2Rwq+KGMPbOvtcvRulaKSvUZkxLtSRcYY+Bfqj5VmluzXHyod5/jzrh0r+N27rWwLU5sw2Maa+Y5xWXFwqzglSdndft1BLsXgw8+YO9agN4SyoLsAASxk9YqKhFNyS1YJNSAUBGsWwWZ4E7A+Q/rNR5cc2e2u1/Q7cffl6ipHBVAFAcNAAoBjE4kKrEEEqDpI3iQKjmXckovS5iMdevXVW8ylQS0jNBzAEIFUbyeVYKsbSzJ3OcQzK1OKVlr66kO4zzlCIXgLMQXOSGEeQJFelhqcHBSb1X9iVGipRg5Pb+x6LgxFtBt4V09wo3dkXuPVw4RuJ4oWrT3WZVVFLFm2AGpJjyqUIuclFbsHy52v47cxuKuX7kjMYVT+qg9lfhr6k19vgaUKVBRg7+vdlsoSg8slqel/Qh2pZlOAciUl7ZaTKfroOhBhh5FuleXx3hzppYhddH9GZ1UTlY9hFfNEwoBI3Pu++gG8b+jf6rfI1KHmRyWzHLew9K49zqFVwBQBQBQCX5eooBVAQD/d3+rc/iq3+tfAnLzCeK4u2iqHdVlrZEkCQtxCT7hXaUJSbyq+/wAmRsQeI8eRL1tcrNEHMuWCLggEa61dSw0pU27/AIiNywt4y2b7JnTNlUZcwmQbhIiZkCqHCXLUraf4F1caxl1lt32UwQ2h6eFOtdpxUpRTIVG1FtFIl9msEMxOW4oExp4GPKtbgo1NOxhnJunr3+hq8N7C/VHyrz5bnoR2Q1jcatqCwY5iBoJ61RWrRpWzX1aWivuSHjcHn8DVwK1muB2ZipX9UQQY85G/nNVQVTPLM1bp/wBge4ZjFdSynTM24gjWflFdp1Y1FeIG8bxQiRaAd1IzLtAPPz5fGoSrLM4Q1atde5GTa2QjG425ZN248G0MoUL7QJjefXr0rspyheUtv7kXJxu3sO4/iVvDYdr9zMERcxgFjB6AetXxWZ2RbFXdiRh8WtxEdQcrhWBIjQiRIOtDmxJrgCgOGgK7F8YtowtnNmOgAHlVLrRzZDlSXLSb67GUxmIYZrZtHumcq7PMwCZK8gRoffWJQULxto3q2UYrGVK2jVlsMYbCm9cCqHWc2jTMLzkCJ102rsYZpWRljF1ZJfMvOznCy+W9dUqUJyKdwBp4utepaMVlhsew7Uo5ImmTYVEpE3L6qVBOrGBvqRrQHkv0y9qpK4C3Ikh722w1RdCdDGb3L1r18Fh8tGWIl7L46NmrBU+ZXivzueQ8QXWeor3ODSzRyvo/mauMQyzzLqjTdhrTpcS6hIZSHkddgvodZHrXp8UnF0pRnsz4XH4uVKacd1t9T6FscWU2u8YMFAWSBPiOhURqSDptX5zdPY97C4iOIpKpHqWNDQJG59330BX8cRntXLaMVbLOYchv15wRpV9BqM1Jq5GpDNB62H+FKVtKrMWZQFJMSSBzgnlB99QqtObaW51QcUk3clk1WdEo4IkEEdRrXWrbhO4quAKAS/L1FAKoCAf7u/1bn8VW/wBa+BN+Yo+2XtWfqv8ANK2YH+r4fU7Ap+IfpbX+nh/kK00v05e8ipkjC3B/aMa/pbvIx7FzntUJr/2nwXzRWvOTrfGxet31yFZU3BqdvAsaga1S8M6Uou/p8yuVRSixjCmbLcvzi/uNUp/qL2+pkf6b9zYYb2F+qPlXmy3PSjshTD8fGuEiPgsO6rFxy5k67VVQhOELTld97WBICAcqtBX4jEC3cVVH6ToNJGmvn+OdVSqRjNQ7gj4hbNx5U+NCA8aGdxmMdQRp6VXOjTq1FLrF9Pbr3EZ7pHcagKsv7Mmd/Jd/SratKNWDhLZ7nU7O6O8Xxy4e0vgZxISB+qCD4mnl/OpQgoRUVsiE5ta2uSsNibdwzbdWCmDBnX7tPnUjqkpbMm60OhrQEPGcRS2yoT4m2006AnoJrjlFNJlc6qi7dTPXL/f/AJxUHeWXmdzHLl8+lYqlXNdxjqiNOlzrNyS16ldjUZ2t2GJXMWeHXRp5Tuo86rWapZPT0KsTSUKnJUrrf3NFw7hzr3Jt3R3ag5wDIY9B5D7K9CnDlxytGymowp5WtS751I4CbCgKztFxu3hbNy65EoshZ1Ykwo97afGtGFw8sRVjSj1+RXVqKnFyZ8u3OIPevveutL3GZmPmfkOQr7HieGy0VSpLSNj0OE1Iwq5pve4zj7gJ05VXwehOGsluy7i1eE3aL2RuOwCHwyP2fsDGo/xFNKjLX8uj834u1n09T1/s3iMvhPstH/Kvx/h3FGsZKnUekm7ejW37n0fCaThRsttDR2rgYSpBHUa19aemIuMZgbmNeQ/r0rqtuyUUt2Iv2wtpx/lb3mDJqUXeSIzd0wBykHkQAfXkeg5/ZTfQktY2JFQIkfhw/NJ9VflU6nmZGHlRJqBIKAS/L1FAV+PxTi1eI8JQEAgzrAMwR5irqcE5xT6lsILNH1K7g2IdrN8MxaFMTGkqxOwq/EQjGccqLa8VGSsRe2rgNZn9l/4KtwCvm+H1KYFPxFfztkyf0eH+QrTSf+nL3kVMtrPDrwxxcocod3LSsZXW4FO87+VZ3Wg8Plvrp80V2ea5XcI9m5/oH963V1bePv8AczrZ+xc8Gw6vavBhsZGpGoQ9Ky4iTjONvzU5TgpQaZpbKwoHkKwM2paCuf486HQoBvEX1RSzkKo3J2rjaSuw3Yy2DsWbbLda4VznwgmR4wdxG0EVSqcISUr/AIyrLGLuTcdeW2rsMhkeLLu0bGOo9T0q6yWpbsVy8dRlG83JnYRG258vtJqGfVKz1IOaViP2s42vdG2BObwxoRBg/qncidSelWo7K9vCd7E38uRO5yZiYMnku+o12ijK6K0fhtr+M3FcLhhr4JKqRmG/lPPz15UjKLdrnJqVtF8SJi3C3La91nknxb5ZPn8ahOXiWhU7Rkla5m+JNetC6mRFtvcGVjoI3Hr90edZKjlG66NmeWaF01pc0HD+DKiEMWYsuVpM78hV9KgoPNfU9CE3GCj21JvDsEtlBbSYE7+Zmr9eonNzldkjnQicTYUB5h9LGPNwd2vspv5tIBPu2+Ne9/D0oSqykvY+Zx2M5mKVJbR+djyrhPZK/dYA+FZ38utfSY/iGGoRdSWr7d2aK3GqdGNo6s9R4N2UwdpQDbDEfrESTX4vxb+L+KVq0lCTpx6JaP4nMNCnWWevNtv1dv7F7Zs2U9lAPQV8vXxuLrv/AFajfu2b4LBUtYxX7C8RigEYxEAn4a1RTg86s+pvweKp1K0YLq0aHs5xVcRZVxuNGHn/AF3r9H4fiXWpeLzLR/f4mzG4WWHquD+BZDc+7763GQTiLZZWUGCQRJ13EV2Ls0zjV0BtysHpy+Yml7O6JRbWpyw0jXcaH8Ea0krM7JJPQRw79En1V+VSqeZlcfKiRUCQA0Al+XrQFXxT9Diff+4laKX6kPzqy+HmiZy1eZbFzKSMzopjmCryK3OKlUV+zL6qTki1w9hb+Htvel2FwKDJBg3VUjwxy0rNKTpVWoaK30Ms/DLQmYrhFg3bRKbabtsi+EROsVVCvUUJJP8AGVPcnR+cf6ifO5VX9K9/sDK8D4XdOdSpX82Ultg02zGnkK9HEV4aNO+t/mZo05O6LnhGCdLT7EvsAfKOcVkr1FKa9CVODjB+pc2th6VmZoQrn+POgCgIHHiv5PczKWEbDryPuMGfKq6uXI8y0IztldynweBuLY8XjC+NFygEQJEHed+u9chFqGuvYjFWjqHcpeZb+Uq6ggKSec7iNTudPKpR8VpWCgpSU3uR8dwtXQK2gadBplI9OY3qZOUVJWZlr2B/Iz4lJLjxMPEAFkAGdJ/pUk242ZVFunDxtF32QxjPfRe7IQKSHPWIj7641oThPMrmwx+IYIxtjMQP5ggeY3rknkjmIyk5aQEcJZXHeBMrHRvXQmqKMozvNLU0zU4xUZDnFMcLKFyCQCBp51ZUmoK7KJyyq9hnB4LMA1xu81zLI2BggRUFTUrSlqdpuWVp6pllVxIKA5zoCv4reC2wDMkjLBI1HPTlXmcWx38pQco+Z6L7/AlGDkm10PL+1vGsPhhNwZ7nJB1PU7RpWv8AhDCcQxdPLBZafWT3fsvqfF1uHurXkoS169kef4jtpfZ5hVX9ldPj1r9Io8IwtKOR3b7u7N8eCU1De7LbA/SEw9sfj3V5mM/hDCV94mV8Jq09aU7F/hO2lq4P6mvDf8FU6L8CX7X+Zjr4fGLSUmTV4t3isoiGDCQZ5GvN4xwKnQwsqnWNntbqXcGdSnxCjmb8y3G/oy4pldrTk5DEwSOcjbpBr5qOIWFrxqPZ6P7n7b/EmEU6aqLdHsK7n3ffX0yd9UfCiqAKAYcZWDcjAP3HafL31JaqxNaqxzh/6JPqr8q7U8zKo+VC3vgabnoNf+tRXFFssyu1yp7PYi5ctsTa7nxvofrTtptJX3VoxEIRmrO+iKcPK8W5LqWncwQSSTPM+vLbnFZ2+hc5X0Mph8Xca3ezMWm0WIMb+ET8K9GVOMZRsup6VWnGLjZdRnC4Vrti7l0KsrGTGgV5iJ61Oc1TqRb7WKqzUZov8DhGtYdUeJFxDoZ3vKd4HWsVSop1XJdvoY5yUpXRYX/0lv8A3fu1THysrFjDKHL+LMQAfE0QNvDMczy51zM7WOiMMrgvIAlpGs8gNdNNvtpK2ljgvB2yqBWiR0/rSTTd0cimlqOpsKiSO8/x50AUAUBRdqeH3LostaTO9u4DBfIADuT12H2+lSi11Ka0ZOzjun3sNX8Q4u913TMuSRc0yyTGWTp1/Gtc6FmbxWsVWNxVy5eNq0pVkIYkwykDcA68wPtrjTsRcm3ZDeK7N3r9xma+uXwsFyTAHtDSNekVK6JSgpKzLbCcFNq8bpuL3OXL3YUiDMTv0+dRc1GNyShm8Ja8N4nbvZgk+HQyI8qrp1Y1L2IqSvl7C7iur5syi0BqNqg4zVS6fhL81Plu+/cW6W7yaw6H4GKslGNRa7FUZaXRIRYAAgAbCp2toDtAFAHOgKvEcIFx87O0QAqiIA56RvJrzMdwuGLkpTk9O1i9V7UnTtvv3KviHYjDXNSgYyJzRtz5b16GDeJwqUYV52XTT7HiT4TRd3BuPsyvvfRbgW5EegT71r2qfGsVBWbv7iPC0v8A7J/v/wBEAfRJhP1rtxZMCO766bpv5Vo/9RYm1rL+/wBzQsFDu/3On6JMMCIv38us62wfKB3dcf8AEOJ7R/Z/c7/I0eqf7jy/RTh1kpicUGjSGtj4xb2qmtxmtWg4VIxafRq5JYKgmnl29WTeFfR9bskML97NzkWiCeevdgn1r53FYDC4hWlTS9m19T0547ESjlc3b1d/ma7C2soyyTAGp351bRpKlBQTbt33MkU0tR6rToUAxjbbNbdUMMVIBPIkaGpwaUk3scba23IHAsLc7hBefOY5aDKdhpE6RVuInHmNwVjlGUowV9y0VANgB/Ws71Jtt7iMPZCiASdSdf8AMZP2k12TuRSsLfl61w6ZKzwx7Vu+WCiLZQwZk+Fum0V6LrRnKCXc9KrXjNxS73H+AfosR9X+FqjifPAoxPmRo7lhXADKGAg6idRzrCm09DINX1bvEIAyiZMxuI0Ea1KLSi7glVABQBQHE2FAdIoDmXzNAGXzNAca3O80Bx7IO8/jWgIOA4HYsgrbUgEzuTqfWut3IQgoKyJSYNQZE7Ab9Nq4TFmwCIO3SuNJqzOptO6E4fBok5AFnUxGvr1rkIRgrRQbu7i7loMCDqDoRUnqRaucsYdUUKugHKuJJKyCSSsheXzNdOhl8zQBl8zQBl8zQAE9aAMvmaAMvmaAbvYZXjMJghhPJl2PqKAcy+ZoAy+ZoAy+ZoAyetAGXzNAGXzNAGXzNAJt2goAGwED3V1u7uwtBWXzNcAZfM0AFPWgGruEVlZTJDe1rvoBy20AqSk0010JKTTTXQaw3DLVtWVQQG9rUmdI5mpTqym029jsqkpbkoL61WQDL5mgDL5mgDL5mgDL5mgOgUB2gCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgCgGMffKWncalVZgD/AJQT91TpxzTUe7Izlli2V/ZbizYqwLrKFJZhAmNPWtGNw6w9Xlp32KcLWdannasP8fx7WMPcuqASgBAOx1A5etV4akq1WNNvcnXqOnTc10M3he0XEbiB0wisrCQQ2497V6M8FhIScZVdV6GKOKxMlmjT09y64BjsXcZxiLAtAAZSDMmTI3PlWLFUqEEuVPN30NNCpWk3zI2H+0XFhhrDXYBIgKDzY7D03PuqGFw7r1VBfH2JYityabmQ+yfaA4pXzqEuI0FROx2Ouu4Ye6rcdhP5eSyu6a3K8LiecndWaJXabijYbDtdVQxBUQZjxMBy9arwdBV6qpt2vf5FmJrOlTc0ihsdoeJOquuDUqwDAhtwRII8XSt0sHg4ycXV1XoZI4rEyWZU9PctOz3aUYh2tPbNq8mpQ9OZEgHmNCOY3rLi8E6MVOLzRfUvw+KVVuLVpLoQeK9rzYxncOi90MsvrmGZQZjYgE/Cr6HDudh+bF666exXVxvLrZGtO5q1YESDIOoNeW9DdcouyPHXxaXGdVXK+UZZ6A8/WtuOwqw8oxTvdXMuExDrRba2ZfMaxGsw3De1mPvqWtYVHAMEgnQwDGp6EV7dbh+Gou1So18DyqeMr1FeEL/EsOH9rH75bGKsGyz6K0yCToN+U6SCdaz1eHx5bqUJ5kt+5dTxks6hVjZsl47jrpjrWFCqVuLJYzI9vbl+qPjVNLCKeGlWvqnt+33LJ4hxrxpW3L+sRrKDtD2ot4ZhbCm7ebZF5TtmPKegBNbsLgZ11mbtFdWZMRi40nlSu+yKx+0+OtjPdwRFvckEyB1Ohj3gVpWBw03lhW19ij+brx1lT0LrDccF/DPewwzuAYttoc37La/bMVinhXRrKnV0Xf07mqOIVSk509X29TPcQ7V4+woa7hURSYBJO8ExoegNehS4fhqzy06jb9jHUxlemrzhb4llw3i3EHuILmFVLZPiadhG/tVmrYfCxg3Cpd9rF9OtiJSSlCy73JHCeOvdxd/DlVC2pgiZOoGvxqFfCKnQhVT8xOliHOrKm1sW/EHuLbc2gGcAlVOxI5addqyU1FzSm7LqXzclFuO5X9mOODFWc8BXU5XXoeW/Ij7+laMZhXh6mXddGVYavzoX69SPxTj7rirWFsKru2rlphF3nTnEn4danRwilQlWqOyW3qyFXENVY0oK7e/oaAVhNYUAUAUAUAUAUAUAUAUAUBD4z/d73+nc/dNW0P1Y+6+ZXV/Tl7Mo/o5/uY+u/wA63cX/ANy/ZGXh36C92Te2n9yv/VH7y1Rw7/cw9y3GfoSM5wLj2Lt4e2lvBNcQLAcFvEJOuimvRxODw860pSqpO+1v+zFQxFaNNKNO67/iNVwHiF68jNesGyQ0BSSZEAzqBzJHurysTSp0pJQnmRvoVJzTc42M9x9xisfaw0/m7P5y50J0MfDKP95r0MKv5fCyrdZaL8/NjJXfOxEafRav8/NxHEHGE4kl4Ed1iBleNg2gJ+ORve1dpJ4nBOm/NDVe35f+xGo+TiVNbS3/AD9v7lr9IH9xufWt/vrWfhX+6j8fky/iH6D+HzI/BO1ODt4ayj3QGS1bVhlcwVUAjQdanicBiJ1pSjHRt9u5Chi6MaUU5apIhcDvflXEnxNtSLSJlzERmMQPfuY6AVbiY/y+DVGb8Td7diuhLnYl1Y7JWI/E+HJiOKXbT7Na0PQhFhh6GraNaVHARnHpL6kKtJVcU4vsTOyHE3s3DgMRoy/ozyI3yg9I1HvHKqMfQjVgsVS2e/o/zf8AcswlWVOXIqbrb8+QfRj+hvf6v8IrvGf1Ie31HDPJL3+hs32NeOj0nsed/R/xzD4ew63rgQl8wEMdMiidB1Br6DiuFrVqqcI3Vvqzx+H4inTptTdtfohzj/Ekx2Iw9rDS+R8zPBECVnfWABM+lQw1CWEo1KlbS6skSr1Y4ipCFPWz3JvGP/V8N9T/APWqsP8A/HVPf7Flb/eQ9vubSvHPTMN2IQXcXi7z63FaBP6oZnB+xQPdXtcRbp4elTjs19vvc8vBLPWqTlv/AJNwRXinqGH7PoLXFcRat6WypMDYHwN9hdh769rFPmYGnUlvf7/ZHl4dZMXOEdv8fcf+lH+6p/qj9x6jwX9d+31RLif6S9zX2fZX0HyryHuejHZGO7Nf+p4z0P7y17GM/wBlSPOw3+6qG0rxj0jB8ZZuHYz8oRZs3wcyjTx7+7Xxe9q9vDpY3D8qT8Udn6fn0PKrN4WtzFtLf3/PqWHYTh7ZXxd3W7fMg9EmdOkn7AtUcSrRuqFPyx+f59S3A0nZ1Z7y+RrK8s9AKAKAKAKAKAKAKAKAKAKAicXUmxdABJNtwANSTlO1WUWlUi33RXV1g/Yw3Zvjd/C2BaODvvqxnK438shr3MZhaWIq51VivivueVhsROjTyctv4P7FzxDH3MVgMQe4uW2nKEIYs0FGkDKCRqRtyNYqNKGHxcFnTW9+nU1TqyrYeTyten7FdwbtHesWLdr8ivtkETlcT7slaMRgqdWrKaqx19V9yiji506ahy3p6P7Fxw/tLcuC4Wwt63kQuJVjmI0CgZRJNY6uCjBxSqRd3b29dzTTxUpp3g1ZFLwDsiuJttfxYuC5cdmj2SBO5BE6nN7ordieIuhJU6FsqS9TNQwSqxc6t7tj3GOwdhbNxrPed4BKgkGY1IgDciahQ4vVdSKqWt10JVuHU1BuF7iuJXr1/hXit3O9BRWUo2YlXXxZYkyIPxqNBU6OP0ksurTurap/4FWU6mE1Tvp0fRlnwns7hnwtoXMPbDtaTMSgDhigkzEhpnXeazV8ZWVeThN2u7a6b/Ivo4Wk6Uc0Vey6a/5InY9r1h3wd1HKoSbdzKcpG8ZojWZ36ireIcutFYiDV3ur6kMHnpt0ZJ6bOxy1h3/tdnyPk7uM2U5ZyLpmiJrrqQ/8eo3V77df2OKMv5xytpYmdsuA9+guW9L9vVSNCwGuWes6jz9aq4fi+TPLPyvf7/csxmH5sc0fMtiF9G+HdLV3OjoTckBlKk+EawRV3F5wnUjkaenR36lfDYyjCWZNa9fY17bGvJPQZhuwHA0aw/5RhwWz6d7b1jIu2cTEz9te3xTFyVVcqeluj9fQ8vAYeLg+ZDW/VenqbPC4K1bEW7aIOiqF+VePOpObvJt+56cYRj5VYxnaprtviFm+lm5dCWx7KsRM3BEgEA6g16+C5c8JOlKai2+r9jzMU5QxEZqLdl0XuWOB7V3blxEOCvIGYKWIaFkxJlNhWergIQg5KrF26fjLoYyUpKPLav8AnYicV4VicNiWxeEXOr/pLfrqYHME66ag9RVtDEUa9FUK7tbZkKtGrSq82lrfdCrna/EuMtrBXRcOniDQp/4iffFFw6jF3nWWX03/AD9w8bUkrRpu5N7I8BeznvXzmv3fa55QTMTzJO8aaCqMdi41bU6a8Mdi3CYeVO85+Zkb6SsO74dAiM57yYVSxjI+sAbVbwepGFZuTS066dUQ4lGUqaUVfX6M1dn2R6D5V5b3N8djz9MZew2OxNwYa9dDkgZVaNwZBykHavedOniMLThzEmu7R5CqTo4icsjd/Rmj4J2huX7mRsLdtDKTmcNGkaaqNdfsrzsTg40YZlUUvRf5NtDEyqSyuDXuMfSHYd8LCIznvFMKpY7NyGtWcKnGGIvJ2VnuQ4hFyo2SvqX3DVizbBEQiCNo8IrBUd5yfqzXTVor2JNQJhQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQBQHCaAJoAmgDNQBmFAE0ATQHHuAAkmANyaA53g60Ad4OtAJfEoN2A569BH8x8aAUt1TsaAbtYy205XVo0MEGDEwY2MUB21ikYSrKw01BkagEfEEH30AvvV60BEfi9gEg3BIJBGu406VJRb2ON2E/wBtYf8AxB9v8q4ot7HXocPHcN/ir9v8q5HxXt0OtNbiv7Ysf4g+3+VOlxZ3sB4zY/xB9v8AKhwcPErXdtdzjIs5m1gRqeVcbsrkoxcpZVuMcO49hr7FbN1XYCSBO0gTt1IrkZxlsyyrh6tJXnFonm4PwDUikO9H4BoA70fgGgDvR+AaAO9H4BoA70fgGgDvR+AaAS19QCSYA5nSupNuyONpK7GP7Usf4qf8hVnIqf8AFlfOp/8AJC/7Qtf4i/EVHlT7M7zYd0Bx1r9tfjTlz7M66kFu0H5fa/bX405U+zOc2HdHBxGydrifEV3lVP8Aixzod0cTiVksEFxCx2UMJPoPcarksrtLRlsIucM8Vdd+h3GcSs2o724qTMZjExvE+oqMpKO7LKdGpU8kW/YkI4IBGoOoPkakVNNaBzoDtAJLUAQaASiwNKAVNdAoGuAZxaAowOoIIPoRQFZct2bcBrhXMTlBuMJO8KJ1idgOlAPLhzuLlwf8T+8tAQUW8xDyCRnCyV0UtvGUawBzoCRbdyGllWDDErzhTuGHIigKDiN+1YtXbveKM7AMVTOysxfbxkBtWMTufOoaR17ktZadizwWJyXnw4XMwVHGVcihYCQCTEysRPKpkSc2MbLmyADqXHWBsDMmPjQGYxYYXLvMd4/L/Ma6lRezaZO9RLXUYMxrt8v5USg34XZ/MPMt1dEQIR5qfsqV8718M116MeVaax+RKI0g+41xN7pa9UcaWz+DOn8eY/H40pv8f7Mbfm6LYoTwzEid1ufuiqqz8DZfhdK8PdGf+jWwyYi4RDfmyI2/XSsuGleTPV4tK9Je/wBGbvjF+4lpnQDOqhsp1BjccuU16NNKUkpHztWUoxbiY7HcaZ4yXbisdSAWAHXL5b1vhRUd0jzZ129mxFt8Qq2x3915eSS5mAAIby513LB30Qz1NNS+wdwqpHeXGkkyzMY9J2FZ5JN7I1Qk0t2PWsSZILtJ235Dz0qLgux1Td7XFu7x7TT1kgUSjfY63K24i4r91AuOCoEHMST6ltz60WXNsJZsujKrB47EM7pcbwG2fDpowGusSdRz61fy4LLKO9zNzJvNFvSxGs4bxsTqPlW+VTwpGJR1JAtEtIIC9POq76epbkUndaIp8BxV76uozLczQrMBAjX+da6uHjSak9V1sVqebwvfozvAsTea5ft3STkIAMQD7UxprtTEwpqEJQ6iKndpkjil/u0lPakCBqYg8vhXn1cVGkryV/RG7BYCeJlZNRVr3e2nQk9jsOTfW5c1cFgDsYynl7zWSthnGGeq80uj7LsbnjoSlysMnGn1Xd9+voWH0hYfOLPln/hry68HK1j3OEVMjn8PqanAj81b+ovyFXJaHk1PO/ce51IgUHG+01u1nRTN1YAEaSRuT5dKwYnHwpJxjrJdD0MLw+pValLSL6lP2bv3hca9cYlX9qefSPMcgOVYcDUq828rty6fX0NXEeRGkoKysbQMIncda908Ua/KrYIUsoY7CRJ9K5mV7XO5WKt4hWJUGSu4/G48/KuRnFtpPYNNajtTOHLu1cBT8S4TbvOjOD4J2I6jQyPtEH41CVOMmm+hOM3FNLqT2feILATEx1iegJG9TIFfgAp7qWh+6k2w+kNBJK84OgMda4DuKvG2HLLKsRB1IlsqwcoLA+6ugr+C4LD4bvFW6GzkN42TSNhlEERvJqEIKGxOc3PcsrGUvcKGRktgANpp3m3Q/wBKmQI+Ms3WtXkdtXLLbK6QreyD1I1k9BQ6ij4gYu3CD+u+x/zHlU04S0cRaUdUyOpjmNfxqK6lGp4Wtuo1hqdJA1Hw/HKorxeGW62Z1+HxL9jhjblv51y7azLdae4sk8vQCfs1H4/HOurWzfXR/cbfAurSzw++B0ePgKpru9Nt9i7DaVo+5T9glKX3LAx3ZGmv6y9Kx4R+N+x6XE5J01bv9GbDjGIC2nczGUeuug+depTjeaR4NWVoNmKxWK3CZQfNYgnn516EYdzzJT7EnA4MsJnff1jcVGcrEoQzalpcfJaljqBv1jb7qqSzS0NDeWOpTf2iuYuk94YGoO3MQdNpq7lu1nsZuar3W5eXvH+bZbo0DZk0GhiAw1nyjaqFpqrGt+JWdyW6SOQ9/wB9QTsTtdEO4kIxIGbK3yNWw8y9yma8D9io4diXzXAImP516NWEbRbMFGc1msMG6+bKYqeWNroqbdxOAXdzuDp+PfXarv4UIaairnEchIPtXNE00nYT5SRWDF1o0lFLd7e563DMJLEOc35Y2cvb0F4Lh5U97cjvSIYg6RygegFVUKNpcyfne/Ytx2KvDkUf0k7q+9/83LfgqH8oQ8vF+6auxLXKaPPwqfNTJna2zmFv/d/DWKhFSvc+mwM8rkXmEHgT6q/IVQ9zFPzM5irhVWYAkhSQBrJEwKhJ2i2hBJySZ5pg8Kz3ouAhicxzCCSTqYPvPur5aMJ53Konda/E+pq1oQo+Bq2xN43j+7GVdB8h5eZr6rAYRwSjFeOW/wBj4TG4mdapkj8CDg+0bm2bDPAAOUQDm55STyFb8bwyVNRalp/V+djRTqcijZq779iut8aYseQOhgD5bGr/APxMOU8rTa6dCmarrx5/EbvswDchycpSNBuQef1T08vKvIqQhKopLRrf1L8JKpy7y2ZpoqZeIunwmBJ6e6uAyfEuP4i3jLeH/Jybb5RJ0JJO6spIAHQ9PMV2xYoJwcr69i7u3GhiLTZiIkd35wJzTAJPxrhWLw2HChTlGYKFzaTAG09KAWHeG0UH9TWZ8I1IgR4pHoJ50B45wXhfEfy5zeZ84JAESe8JBzhv1V5zMQYjlVM08ysj0oyp5HdrLbRdbnqWJF03bTBQk5lY5kkLlDSfCQTmBETsJncVceaP37TSssxOcQpywZkHZRsuY+6gM9i3VrtwA6h2HvzGjVWlr0ZYsk9OpH0joR9tM0qTzPZjKp6LdDYvAyOY+VWTjyrTXlkRj47x6oWx0mfIn3b/AAriirum9t0G3bMg+/5/iPjXPLddtfgN/j8y9wZ/8G96P8hVGIVqckW4d3qx9yD2McC80kex/EtYsF537G/iD/017mj4xaD2rg3levvkGvVpu00zxasc0GjznhqHOZzMFzBmJkBgIg+derNq2h5EYu+poOG3d100P3VnmupppvoP8cvlLJKrmaRCzE/EGo0o3lqWVpWiL4TYBSYgkAkabkaj3Vyo9RSirFiiQJkzVbZckI7wHSdK7Y5dEd7w2jTK0nppU4rVe5XKSs16FRdw6kzJ1r0YzklY86SV9CPjGuJby4dQ9wHZuh3MyPKrKeSU81V2j6HVe2VblPxrGvbvW3hfCpkcuY6+dQnWiqboU9akvKu/xNeDwbqvn1dKUdJNbrtp726EzhZt3MzoSWaC45KTPsyPX4VRRwk6Dc6q8Uum+voWY/Hquo0qXkhdJ6pteo9xLB2blsWnd1gz4d+fOPOttGpUhLPFI87wpWb1LHsNg3tOFcQSzEag6ZfL0rNxKpGpG8fzUvwcJQqJSNXxezmy+/7q8uhLLc9+jK1ydYHhX0Hyqh7spe4PM6R51w4ZLjX9+Qtt3Y323b768XHp85//AJ+p6tBr+Ufv9DJ9oGGb/jt5Cvr+Gxbr3XRHy1JXrtmYvmCYma9GtmU2enHWI/wywzPlHtH7TyjzqhVeVByqbbP4kKlrWfXQ9Z7LcLu2c/eCNFAMg83J29RXgPzXI4em6dPK+5odaFwgzQC7iAggiQeVAQbTRKsdVJ35ruD8NJ6g0ALfVtNdf2lIDemYa+6gKTjHaRcOxtrbBCAZiWyBdAQohTJiPjVU6qi7GzD4KdaLneyMvf41curM92o1ugEiSFnVvCSAMg16HqIpq1ntE3YPAxV5VFsxOA4wbZtlCzI7LIBDDQjYzCtuJGm4rlKpJSsyeNwtKVLPTVn6bG0xt8uwFvLIykMXjQkh1hTOYKAROmtazwjLYiyO/ulTp3lyR5lzI+NbqeOhVhymtVp+xGeGnTan0f1E4osoUjbn7/x9tRoRjXzUpfATvTtNDN8wVce+qMIs8J4We/QvruzjXj1Jls9dv5a/caqV3ST6rQOym+zC7sY9fx8R8KnHXK/gQatdfEvuEH/wru277egNU1Xen8CymrVV7iOy6g3GkT4fvFZML537GrF+Ve5o3GscoFbzzzz7tzeuYe5nRPzZAnQkEzp94j0r08LacbPc8vFJwlpscwjW2KXlHhZARB1B5z5xp7q7K9srIaXzIa7QYibZug3BlEhcwj12mffUqSs7HKrza6i+H9pg9gZUIgDMQw5gakR7/dXJUPFuSVfw2SNFw/GhrObNImPlp61mnBqVjTTneFxvF3fCSNOW405TUorU5OWl0RMSp7piGkm2+oPODsPUVOE4wmnLRJoiqcqitHVtMr0w2e1a/OMGXUxz8jrXoKtHNKSSaexjq0XC0ZaSW6EYzGlHOQZ308MwY61jr4jLHJTWaXb0N2BwPMfMrNwp/wDLpfsM3sJltOR+eubqrbmSJAJ5bmp4LDcuadR/F9DuOxyxHhprKrWstperH1txaQlRaYr4gNNY2MbxWzNebV7o8uUbJdBq9wnv1A71l1nMN9NI3qaxHKl5UxCnmerND2Z0uqM2b2tT9XavOxmsG7WNeEl47bmpxDARNeZFN7HrXsOW9h6VBg6aAjY3Dd5bZNiRoehGqn3GKjUgpRa7koyytMwPHOFMx1UqQASvP1HXp7qvwmKqU5JrSVtfU83E0J0Zc2nqjNX+GNGaNOR6+de5HidGatOP11EcWl5k0aXsX2ezt3jeyrQQT5A8tenSvOxmIjK0KflLo2qyzX0XzPRqwGoKAbukhSQJI1A6wNq4BFvHIWyGVbowKz9UnRvdNAO3rCtGZQY2kUBzFWA6lduh6Eag+40BkMfwjvrtx7boGBAuo0+FwoEggbEAaVRUpKbumb8Lj5UI5bXRXWeANcUth5eCRczeHM4jW3OkZcogkbDnNQlRUl4ehdQ4k4yfMWj10LDgHZR1dWuKERSGyyCzMIInLoBIB31ipU6LTvI5isfGcMlNWvubK5bBBBAIrQeWebpYIxF+dmu3T8XY150W+c7Hr1nGVCPsiTeQMhH4EVvozcKqkedOKlBoh27BKQf+tdK01nlxiqR6lUNcO4MkldB8PgajBeOUTkvKmDD7Qfkf5VDZP0ZLdr2NDwlB+RXB9f7qrqLzHYvVMb7O2Abjbjw8j5iqqUUnoi2rJtamgZYMdAKvM5Gx+CS8hS4MyncVOE3B3RCcFNWZkuL8LuYcAWMO123rpbyjKIknKTrryGuta4V1LWT1McsM09NiBirRvYVIVkLICVYarI2YbyDyq6ErSuU1I2SRU8NsAMbE/owpbKY5SoYee/vq6UtLlCi76ml4TiJW4sZAGgwN5VTPprHurNUjqmaaUtGtiU+ERsw1AI1AgCSQZ9RUczRNwTuMcPdVBsnktzeNtT99dnHO03s7HKVR0/Lur6lRxR2WO6JJnUJrA842q2WJhTSpRjfpp/T6l2GwUq8nXryst9f6/RMf4TYzKGcTc1kkeKJ/6rtPDch3k7y/5FeLxzxHgpJxh/x6X7j+AuZpJtlCDAn51rqrL1uedB+gABi4dh/lnlvt9lNknFe51LNe7JVi0FtghwfIe+qpycpvQmoqMFJMk8Dur+UIFiCG2P8AlNVYmMuS2y3DyTrKxecdeAnv+6sWGV7no1Z5bFjh/ZX0HyrNLdlqegvnXDpwigI+LwSXIzDYyDsfT0PSoygpWudTdmu4xiuEWX1K5T1UlfjGh99dSs7oi0pKzQvhuCS0DlLGd5M7VK7e5yMYx2RMoSCgB64CNxFPDIGoI1AlgCRmK+cTQDmDL5fHvOkxJHIkDQH8abUA/QFdjrNxSz2UR2bLmVmyTBjMGg65eR3gaigO8EdzbJe21ti7+FiCdWOvhJ0oCwoANAed4jGqb91RuHcfBjWl8PSgqnf6nP5l3y9jisYPvqrkprM+n0Jcx7CmEaA7/wA4qKfMTm+hJrI8q6gq6xO0/dXM7iuZ1Yy3eTohJ5+Wn71dte1+urObfA0PDdMHd8i/yFEs9r9TjeV6dBnsyWNxtR7PTlmGldlRUNUc5jlozR3N/hUAJoAoBm/hUeM6K0GRIBg7ffUlJrZkZQjLdCbmBtshTKAp5AR8qKck73OOnFxy2M8/Cr+Q3WyrdXNmySQVBOqgydVgweZrUq0b26GSVCXm6ifyq1ctKQZziRodfdE71JJpkW4uJx+B3VW5dzDOysBOsSpjSNNahOs5LLT0fT3LqFGMJ56yvHqvQr+GYHKWJ1cgZiJ1NaaVFU1nfme77leLxUq75cf04+Vdl7isTYuoc1sgctp9dxVsHKVXxu8e3qVt0oYdZIvmX36W/PQ5w83IJuNm15CNPgK0VVC9oaGOF95A1tbpMKQV6zz/AOqKTprVnHFTbyj9nDACCPnVcql2djT7kjszbU31ZVIjMNZ/ZNV4yUuU02W4Rf6qaRb9qbmUW/8Ad91ZMFG7ZqxsrKJc4X2F+qPlWOXmZtj5UOc6iSO0AkrQBJoBuydNooByK6BQFcAm5QASPKgO5h1FAGYdRQBmHUUAZh1FAGYdRQHCw6igPI8HYc4zEkjTv70f+40V7letCGGin2XyM0IOVR2LivIxFXLSv3NVGnmqJEXPNzTkDUVHk4G73kyx/wCpibLZEpLmhP431+VcnB+GHoRjLeR07ep/Hzrr0fsji1Xuy94be/8ABvNGxufIVOnG84x9iub0bInZHFFrrwuoQnf/ADLWnF0HTim31K6c1JmqLEnUR/2awFwUAUAUAUAUBR8P4Rct4u7eJVrbgBBqDbjcZdtSW1HQVfKqpU1Hr8yiNLLUcuha45oRj5GoUleaRZUdotmat4u6WYW8untT08vtr1XTgks3wPMjUqycstvUft41ojT4VB0lclHEysVN/ikXVtr+tJ1HSefurXHD3g5voZ3Xado9R29xa3ZKByc1wwIE6yB7t6hHDzqpuOyLKdZQ1fUnPi220j0qhU0TlipX02HuBYh++VdMpmf+JqGKhHlt9TuFrSc1HoQPpQ4j3K2PPvPsCVo4LR5rn8PqOJNpR+JsuHNNq2eqIf8A4ivGqq02vVnpQ8qP/9k=',width=400,height=400)

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxAQEBUQEBEWFRUVFxcWFhcVFRgWFxgXFRYWGBcVGBgYHiggGBolGxUVIjEhJSkrLi4uGB8zODMtNygtLisBCgoKDg0OGhAQGy0dHR0tNy0tLSstLS0tLS0tNy0tLS0tLSsvLS0tLS0tLS0tLS0tLS0rLS0tLS0tLS0tLS0tLf/AABEIALEBHQMBIgACEQEDEQH/xAAcAAACAwEBAQEAAAAAAAAAAAAAAQIDBAUGBwj/xABJEAABAwIDAwkFBwEEBwkAAAABAAIRAyEEEjEFQVETIlJhcYGRobEGFDIz0QcVQnKSwfDhI1PS8RY1Q2J0gpMXJCZUc7O0wsP/xAAaAQEBAAMBAQAAAAAAAAAAAAAAAQIDBAUG/8QALhEBAAECBAUDAwMFAAAAAAAAAAECEQMSE1EEMjNxkSExUgVBoSJh4QaBscHR/9oADAMBAAIRAxEAPwD5bia9UPIbMCNBO4LYzBYggHlGiRNxET3XUBgqjqjsr8ktzC8TFt3YpVNkYiYdUEmNXG43LpxMWuK59Z92nDwqJoj0j2W/d2IEE1WgHq9bLJjKWIpta7MHB2mUA+NlN+wqw1e39U+Sz4zA1KDQ7lN8HK42Kw1a/lLPSo+MM5xtUWJ8h9Evf6nS8gqHOJMkyetRTVr3nyaVG0NPv9XpeQ+iPf6vS8h9FmQmrXvPk0qPjHho9/q9LyH0R7/V6XkPosyCmrifKfJpUfGPDU3H1el5BS99qdLyCxt1VisYtfynyaVG0eGk42pGvkPonTxtUmJknSw+izQY6lKiSHDLroO+37q6te8+SMKjaPDX94u6PH+m7+QintI5YLJMHnTHYYiLIZUriwExbefh5tr9X8ujCYiuzM1jZ52ZwLZgn00TVr3lZwaPjHhL7xI1p8dTxuBpuCTNoOk8yd+ugBk7tIUmVcQ3RnDdPGmJvbSO5U1DWLoIhzg4cCRlMiez9k1a95TSo2hYdpHoR3+eir+8HzMWi464108loGJxIsAJJ6iZIzbzwHkoMxWJe3MBI4gDcZ4ymrXvJpUfGPCLtou0ymd/8hVDaD56o0ga8dOxM065cCWyQMu7Qc29/NOvVrNEuEddt9x6Jq17yulh/eI8QtdtOPwH+f0WU46qdD5D6KTW1gcwF4Im1xom/EVhAJFzG7fxhNWveScGj4x4VHaFXpeQ+iX3hV6XkPop18LVJu3qt1W/ZV+41Oj5hTVr3lNKjaPB/eFXpeQ+iX3jV6XkPoq6WHc6QNQp+41OHmmribyaVG0eD+8avS8h9EfeNXpeQ+iz1GFpgqCate8+TSo2jw1/eNXpeQ+iX3jV6XkPosqE1a958mlR8Y8NX3lV6XkPol95Vel5D6LKkmrXvPk0qNo8Nn3lV6XkPounsqu57CXGTmjSNw+q4IXZ2J8s/m/YLfw1dU4lplz8TRTGHeIRrU3PrZGm5gC8bhZZ6heCQSZFtSdFZi55UxrIjwC14PB4jM6m05SNZI36Lmxeeru6cLkp7H90VzfMOM5im7YVUfG5otOpKsr7Nxe90kxPO03Bc/HUatI5XuNxOpIWDMY7ZtSkQDeRPNusZCtbiHjR7h2EqsmdVRFCEKAQhCBBWKtWhWENk3AUVOmyVEhUbG06s81wPYR+IZv3806NOvyhaxxzubJhwuB/l5KunQkAipFgTrbnFo9EPwpDmjlBLjEyYb2ncLqs59mg0MRPx3M776B3Dfw4gqrEUa7bl033Gd2aezm+Sk/BAWNdu7QyIBgnXdI8TwVVXCNAtWaTa0jeYN5sAowaaWErzIqAG4kk7iRvHWfFVVcLWYPmA6CA43/DbuhQpYSkfirAabgdR28bd3YiphaABitPDm7/AKILKdCqQHCrwMEngDpwm3cq62HcTDqoMnrjUf4p8VXTpUo51S++AeO63CPHqQadDpu8Oo9XGPFAqbXGRykQYubW/wAkPocag/nf1p5cP0ndn7acE8uHiczp/wA/6Ksro0KZLS41csGP5dJlEmSakXtJ1HHVVvNORlBiTM8NykXUeDlEsbqbQfmTG8Ht/ohzBFqkncJ61W17JktMRp1qw1aO5hRCdQpxPK37FkKurVGEc1sKlQJBQkgEkykgF29ifAfzH0C4i7exfln837BdPC9RzcX02fHfMd3egWihh8Rn5pcHObm11Has+O+Y7u9AraZry0tLpIytP7BacXnq7t2FyU9mxuBxh3u/Uqdo4LERnqmcoiZmwWqnWx0ZRNrXibqtuDxZa5hPNdcyQtbY4pSU3CDCigSSaECQhCBKxmigp00hE2jVRcIMKTRe6TwNyyEUIQgSSaECSUwwnQE9gTFF/Rd4FQVpK1tBxEhpI0TfhngSWxHZ2Kl4UFCmKZPVYm/AAmfJM0XDUeY6vqFFtKpCsZSLtEuSMwba37ELSrQr/d9+YKDaUmCYS65ZVIVzqbQDzpsqUSYsihNJEBSTSQC7WxPln8x9AuKu3sP5Z/MfQLo4TqObi+mox3zHd3oFKnXqhrYJhptwBKjjvmO7vQJMxLwwsBsTJ7QtWLz1d27C5KezpivjSJAcQb6LLjX4lomoXAE2VlLa2JdzGuPYAq9oOxBaDWnKTaeK1tjnkyktOGwNSoJY2QFRUYWkg6hBBJSSQJJNCBKVNRUmapAsTfG7r/oolSeRw/kLJE8M9gJztzCI75F/VXHE07w3cR8DBrcG28aeaxpIk03bvfxJ5gi8DSJy+dj4qNbHBxJyRmBaYO4kHhrI81jSS6ZYajjSG5ALDQnUXmbIG0ag0gd386vBZUkuZYWGud1puY3kgg9libIdiXnV0zO4b9b7lUhLraDDyCDOmiXKO4nxSSUUy48ddVElNJAkFMNPBWMwz3CQ0kcVLwWUoK0OwVQatKpqUy3VWJiSyCSZSQCSEKAXa2H8s/mPoFxV2tifLP5j6BdPCdRzcV01WN+Y7u9As60Y35h7vQKhasXnq7t2FyR2bcLXywWUucPxXKvNXEvbzmZhulqez9rPp0+TFMEcd6m/bVd0hrYG8QtTYTGYsDmsyidAAFlqbLrky5vxHWd6nW2ziCbujuWf7xrdM6yqI4zBPpRnETpdZVdWrufGYkxpKqQRTAQV6H2Gaw4h7iWgtpksLogOJaJ8ytmFh6lcUe12vFxNOiavezi+4Vv7mp/03fRQqYaoy76b2ji5rmjxIX1TZD6gphteqx9STJYRBE24bupacbQbUpuY8S1wIIPYvUj6VFVGaKvMfy8ufqk01WmnxP8AD5EpZrRG5VMeCBe6uY/dE/1XkvXVKYouIkA7vOY7dCokK4Yt4iIsANOjpqixb7q3UXASWkARu46ei1HZNa3NBmIIIi4JHoVnqYl7hBNj1DjPqj3uppyjrQBzju0WM3+x6Lzsx4+LUgkBozEiAQBB1M6f7pUn7LyzmqNngLzad8WiT3da1eyvs9iNp4kYXDuaHZS8mo4hoawgTYEmM1hC6n2WbNo19sUaGIptq0/7WWPEtJZTeRIOtxKnruejgHA0gYdXaLxIgzfWAbWI75QKGFFnVXbrgT0pEAflV3tfTazaOMY0BrW4rEBoAgACs8AADQAblyEyzuXbagwwBDcxsYOkGGxIjSc3ks7HMbIIzXsdLdiqymJi2nfwTbTcdAf5/kraxdZy7dzb8fD6Je8bg0KIoOnKBcRvG+Iv3hWe5P3wO09RO7sU9DPJ/eD8uW0dig3G1AModA4QFM4K8FwB8v5p4qqtRDRZ4P8AAp+lMxOxdQ6vPiqnOJ1KSFlEWEUIQqEhCFALtbE+WfzH0C4q7WxPln837BdPCdRzcV01WN+Ye70CoWjGD+0Pd6BVZDwWrF56u7dhckdm3Zu0+RBGUOkyJWihtd5JLaYJPDqWbZr+TcS5mYQuk3argeZSAWqzY42Iz1Hl2UybxCzOaRYiF6R2PeTmZTAsJXNxGEfUcXkHnHgqOYkujU2cW/ECFD3ZqWHovYLZ9J7alV7A5zXBrcwkAZZJAO+/kvX+5Uv7pn6G/RcH2FYG0qkdP/6heoweHNas2iHZZa57nCCQ1paIbNpJeLncCvfw8bB4XgtbE9oi8vn8ejEx+LnDo95m0eGZmEptMtpsBGhDQD4wp13hrHOJgAEk9QC9J/o1Q6VX/qFToezuHa4OIc8tIID3lzQRcHLoSDcTpqvDq/rHgopnJRVftEf7d1P9PcTNUZqqbf3/AOKtm7Bw5wlKjWw9IkUabXgsbqGNDrxMzN18O25hBh8VXoMktp1XtbOuVrjlnjaBK/Ri/PftdP3jio/vqnqV8x9Dxa6sSuJn0tf8voeNpiKabPTbA9ncDQ2S7bGOpPxM1TTZQDzSYDnLAXuFzJBPCNxW/wBndmbL27Tr4ehghgcXSp8pSNOq57HgGIeHRPOLQbTzgQVY+k9/seIaXEYguMAmGiu4lxjQAb1T9geEd77WxhtRo4d7XvPwhznMdE6WaxxPC3FfSPPcjYvs7hn7Bx2NrMIxFGuyi10k5Bnw2YZAQCSajxJ4rN9mexsJjdp0sPXa6oxzahLSS0c2kXatM2cF6fYTHYz2d2uKAL3OxhrBoEuyZ8PVmNfhY7wXE+xCm522aTmtJa1lXMQCQ2abgMxFhJtdB6z7J6GEobZxeHp05qNfiW0353f2dGnVDeSyyQ64HON7da5vsDisBV29hzhMPyECuHNFR9QOOR5zy8W4Qrvs0P8A4oxn5sd/8gLz32Qgjb9JpsQcQCDqCKdSxUslnU27tT2fw+0cQ2rgq+JecRWNeq+qWNY91VxqCnTaRma0lwvcxvUvaL2BZT2vh8Bg2/2WJYKjXu52RjS/lZOroaBE65mjUSvB+3H+ssf/AMViv/eqL7rtXaVPD7d2XyhAFXC1KQJ0Dn5S3xLQ3vSYLXfP9pbZ2Hs/EOwLdmuxFOk406uIdXcKpcIFQsaLWIjVswjbnsCKe1cLQw9d3uu0TylOoQHuaGjlHNvrYtg/715i/kfbvZtWhtTFUXsOd9eo5gAMvFV5cwtH4pzAW32X1/F1RQxvs/s1xBrUGTU4tjD5AD2lr/0pMQWeJ2ttPY+zcZUwf3Ua4ou5KpWq4l3KOiMxa1oyi2gt3Ln/AGsezeHwGIovwjn8jiaXKtY4kllxYE3IIIMG4uuR9pf+tsd/67/QL1n23abM/wCEH/5or5eUIUmsJ0BWQghWuw7gJItxVnuTomR4qXhLsiFpZhwW5i8Dq3pUxTAkm/BLrdmQtL307QDreUhiACSGiOClxRC7WxQRTM9I+gXKGJcJIgT1LrbHeXMJPSPoF1cJ1XNxXTaXi6iVlxdQ5yJ4egWcuPFasWf1z3bsLkp7Orh6rGuBfcLc7a1BvwsHevO0iMwzaTfsXbGKwLTamXdq1tiz/SED4WDwVNbb9Q6ttM6KD9rUi3LyI6vFUbQ2jyrcgZF+9BDF7WfUjMNFkOKKiKLjcNPgqig6uyNv1sM4lsOa74mu0MaEEaFe++zbbj8XjXlzAzJQdoSZzVKXHsXytdv2R9oX7OxPLtbnaWlj2zBLSQbHcQWg+W9auNqxsXhK8CmfSY9vymFh4dONGLMesfd+gq+bKckZt2aY74uuWcVic7qYqYYva3MWAvzBp0JbMgda8l/2t4X/AMtX8af+JcPaft/hc9XEYTBuZiqrOSNWo4QG2vlaSC6zeGgk2hfKYP03iL2qo/x+fX2etXxFHvEnU+1rFOaQzD0mEizi5zo68tp8V4WrWc97qj3S5zi5zjvc4ySe8krPTEWVi+qwOFwcC+nTa7zK8Suvmm765sza+IwXsoyvhqhp1G4lwDgGmzq7gQQ4EEX0IXidvfaBtPG0Th61cCkfiZSptph+/nEXI6pg8FzTtzFe5nAmrOHLhU5MtFnZi7MHRIvqJi65S3WYOn7P+0WL2fUNXCVjScRDrBzXAaBzXAg6m+okwRK+ufZ97S7Sx+NbWqViMJSD6lYik2lRM0nNAz5AXEPId8RsOqV8RpVCxzXtMFpDgeBaZB8Qu7t7232ljmcnicU40zrTaG02H8wYBm7DKTF0WVfaMU8fVxuGLmOdWq1ab2iDFVxcfiO+YgiIJ7FfiPbzG1MU3GNLRXY17GPbTpSA8c4luSHHW5GhPavLMDfxEjsAPqRC0nHkABoADYAN5MTrBg6nxWNkse0KtbE1H1XtJdUc6o8gauqc9xtYSSTAA17Ar9te0GLx76RxdblDTAZTORjMrSR/dtE6C5vZYvfKkQDAEaAbuvXh4BVPqkiIHc0D0Cyi6+r3GE+0HbDKYZ72wxzWVKlGm+q0AlpOZwudJzTquBhNp1m4oY4Yo+8yagqOyvdmLIk55bvIgiwjRcJxm5SKlpLS37Uxxr4h2IrO5R9RxdUJAEk62aAB3cFq297S18aKPLvLzQaGUzlY0NYI5vMAnQXMlcVSNN0TCuUsvdjLQGAeu/6qoYlwaGzYdSpKFLQWhN9ZxsSYUCSkhVSKSkWHVRQJCEIBdrYfyz+Y+gXFXa2H8s/mPoF08J1HNxXTV4z5h7vQKlX4z4z3egVC1YvPPduwuSnsS7bKeCa0EuJMXHWuIu1gsHhTTD6j4O8StbYt9+wbBLKckcRqqXbcaDzKQF96td7iwjVw3qp20MMCC2lEE94UFX304AwwCZ81yXFbNpYttQgtblgLEVQkIQgSSaSACtVStVhE8pImTp6KtWMZPiqyiErGV3AZREX3CbiCJ4Qq1fhcRyZJygzx759UWFb87zmMku3xqRbvTbhahvlMaybCOMndZTdinue1w1BOUdpJ/dazQxJ+Ilo6yOkNIv8AinxV9En9mVmAeeAnieokG3GCp+50xJNQQCYadXAQRF9CDqpVcA4NLn1B8JIEyTGmu6D5hY2hktk2Otoj6pJEJ4htPRp0Bvfnc+0cOaQe4oo12N/2cyIMnzA7QCk6owfCNx16x9fUq5mODWBrGwQdTB3gi47FIZT6JNxriZp0gAOjO6DHl5qFRtV5ggDt4XAB37yEYflXSymMt5Inebfivom7DvPxVY7Sf4b+iqKW4O8OcG2Bndfd2o5KiNXk9ipxAAcWgyFUQoNFSpTBlrZ7bKTsYPwsAWRCC3EYkvVCZSUCQmkgF2th/LP5j6BcVdrYfyz+Y+gXTwnUc3FdNDGfGe70CoKvxnxnu9AqFqxeeru3YXJT2IBdLAbOa9pdUdkgjXes+Cw1R5mmLhdFuxcRU+NwEcStbYy4vZ4DzkcMnElWYkYYUyG3fHmtlL2ebEurCN8blh2jh8MxpFN+ZwUHKSTSVQkIQgSSaSKCrG6KtTYkIsY0HVReL2TYBN0PA3KogkmkgFsIqvGZzzG7ndIR/RY1ppUMzcznwINrzzYsN29GVKo0JPNg6akAyRJ13BWsFJredd/Ac5sWiYPUR3o5Kk3V2a+gPWOE7rp4fEUmZpp55PNncOCEwYxjR8NPjExYZiYHZJ8lXWok/wCzIcb66SeG4SVZ95vFmgDzvAbPVYBZq+Je+7j4W4WtusiNLcBV1zRIF8xvawKi7CsaMxfN+43E9ehKxEpILQWAGxJvB6txUXVpaGxw8ifqoKKBITSUAUkykgRSTQgS7Ww/ln8x9AuKu1sT5Z/MfQLp4TqObiumrxnxnu9AqVdjPjPd6BUrTi89XduwuSns14LEVRzaU34BbXbPxbrmb9cLHsvGGk/MG5upbm7SxbiYaedYW0WtsKlsSp+KoGg9a5OLo8m8skGDqFuGz8Q52QyN9zaOKrx+zjSaHF4JJiyo56SaSISEIQJJNCKSkxRTZqiLApVMu7rUVJ8bur0vqskVpKSSBIW7D7OL2B+YReeMDXvsVCvhWsIHKA8fK0Cbx3daWS8MaANy6D61BvwNJI0JHAyNeq2igzGOiGM3cJNiTujjHcrYuztwlQ6MPpun0Vo2ed7mjj4qbGVjacu7WN4bu4WCrp4YGeUflgkcZjgSlkuy1WgOIBkAmCowtmagPwk9/rp2KbcWZ5jBPXu1HoVJhlHqwvpObqI1HeNVbSwZdqQLTcqZbUdrHl/PwqmvSyxeePaDp4QpdlaY914w1MGHVJ7NN/0His9dzNGjv6lUpsoudoCUlirShaqWDcW5iQB2/sipSptB52Y7gsbwXZUQrHOGaQlyhGiyVF1MjVdnYzYYb/i/YLjOcTquxsT5Z/MfQLp4TqObiumrxvxnu9FQtONpkvJAJ7jwVHJu6J8CtWLTOee7bh1Rkj1+yzB1+TeHRMLrDbGIqc2mzrsLrl4Pmva5zCQDMQV0q21N7KJaT1GFryzs2Zo3L/vb5LjH4ZNtdyKmwXhpLn3AmFUMfiXWG/iLeak3B13yXVQJ15yZZ2M0buMUldWw7mkiCY3gGFDkndF3gVcs7JmjdWhT5J3Rd4FHJO6LvAqZZ2M0bq0KfJO6Lv0lBpO6Lv0lMs7GaN1aBqp8k7ou/SUck7ou/SUyyZo3SU5bHXCXJO6J8CrGNI/AT/ylZZZ2TNG6hJWck7onwKXJO6LvAplnYzRuvwtDODL4aI1NpJ19UMZS5RoJlt8xmL3jsGio5J3RPgUNpOm7T4FLTsuam1l9Z9IjK1pAEXFzMQddxspnG1XGGgCdLRuAt+nyQasfDSI13EfiBbu3Rx3pFlWoQdImPwxp9QrN0/RHvKDqdQ/E477DrE7rXhRqU2CJfm/oRMd0qb8E/M3ObEwSJdA4qx2Hpt3Of/ynrB4dR13LXOb92WpRHszGrTHws7yrGcs6zW9fj19yMNWfTECmSb3LToY6upRqV6xJgOE8Gn113lMtWyTiz9piCrYV4BLnARun+cSk6jSa0EuJLh4WPDrVLqbzq1x7io8i7oO/SVclSZr+8r24hrAAGAneTv1+vkq34x53x2KHIv6Dv0lLkX9F36SmSdkvSg55OqirRQf0HfpKvZs+odRHasooq2XNG7GhdRmzANSStDMK1ujfJZaVSakOMyi52jSV2dlUSxhDt5nyCKlUN3OPY0n9lbg6uYE5S28XEHQXXTw1Fq3NxNV8NehCF6DzghCEUikhCrFIIQhRQEIQgEFJCQGkhCoaEIVAhCEDCEIUAkUkKgCaEKAKSEIBCEIBCEIGkhCQBCEKoE0IRX//2Q==',width=400,height=400)

Kaggle Notebook Runner: Marília Prata  @mpwolke